# Imports 

In [106]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import re
from sklearn.feature_extraction import stop_words as stop
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 10)

# Functions

In [107]:
# used to mostly make the accuracy scores more readable
def make_percent(num):
    return '%.2f'%(num * 100) + '%'

# go through 'columns' of 'df' and replace parts of text based on 'regx' with "".
# used here to remove syntax from each headline in the dataframe
def df_regx_replace(df, columns, regx):
    for col in columns:
        df[col] = df[col].replace(to_replace = regx, value = "", regex = True)
        df[col] = df[col].str.lower()
    return df
        
# go through each column and tokenize/un-stop-word it. 
# used to begin the proccess of text normalization
def df_text_analyzer(df, columns, analyzer):
    for col in columns:
        column = df[col].copy()
        for index, val in column.iteritems():
            column.loc[index] = analyzer(val)
        df[col] = column
    return df

# remove rows that contain nan values
# used to remove 3 rows from this specific data set
def df_text_remove_nan(df):
    for row_index, row in df.iterrows():
        has_nan = False
        for index, val in row.iteritems():
            if(pd.isnull(val)):
                has_nan = True
        if (has_nan):
            print("removed row: " + str(row_index))
            df = df.drop(axis = 0, labels = row_index)
    return df

# may not be needed
# create a single array of all the tokens that occurr in the data set.
def df_text_single_array(df, columns):
    element_index = 0;
    for col in columns:
        column = df[col].copy()
        col_num = 0
        for row_index, row_val in column.iteritems():
            for element in row_val:
                element_index = element_index + 1
    array = np.empty(shape = element_index, dtype = "S30")
    element_index = 0
    for col in columns:
        column = df[col].copy()
        for row_index, row_val in column.iteritems():
            for element in row_val:
                array[element_index] = element
                element_index = element_index + 1
    return array

# may not be needed, used to iterate through all cells in a df and transforming with
# tfidf transform function. Was having memory? issues.
def df_tfidf_transform(df, columns, tfidf):
    for col in columns:
        column = df[col].copy()
        for row_index, row_val in column.iteritems():
#             print(tfidf.transform(row_val))
            columns.loc[row_index] = tfidf.transform(row_val)
            print(columns.iloc[row_index])
#         df[col] = column
    return df

# append all the columns into one column, used to combine columns for aggregate vectorization.
def df_concat_str_columns(df, columns):
    concat_columns = df['Top1'].copy()
    for row_index, row_value in df.iterrows():
        single_string = ""
        for col in columns:
            single_string = single_string + " " + df[col].loc[row_index]
        concat_columns[row_index] = single_string
    return concat_columns

# Parse the data
Read the data into a dataframe, vectorize the first headline column, and extract the labels column.

## Import data

In [108]:
data_set = pd.read_csv("C:\\Users\\carmi\\OneDrive\\Documents\\Datasets\\stocknews\\Combined_News_DJIA.csv")
df_original = pd.DataFrame(data_set)
df_original

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as countries move to brink of war""",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)',"b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'","b""Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing""",b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.',"b""Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side""","b""The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.""",...,"b'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?'",b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to prevent an Israeli strike on Iran."" Israeli Defense Minister Ehud Barak: ""Israel is prepared for uncompromising victory in the case of military hostilities.""'",b'This is a busy day: The European Union has approved new sanctions against Iran in protest at its nuclear programme.',"b""Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia's breakaway region of South Ossetia""",b'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report',b'Caucasus in crisis: Georgia invades South Ossetia',"b'Indian shoe manufactory - And again in a series of ""you do not like your work?""'",b'Visitors Suffering from Mental Illnesses Banned from Olympics',"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,"b'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?'",b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli training, we're fending off Russia """,b'Georgian army flees in disarray as Russians advance - Gori abandoned to Russia without a shot fired',"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zealand Passports doing in Iraq?',b'Russia angered by Israeli military sale to Georgia',b'An American citizen living in S.Ossetia blames U.S. and Georgian leaders for the genocide of innocent people',...,b'Israel and the US behind the Georgian aggression?',"b'""Do not believe TV, neither Russian nor Georgian. There are much more victims""'",b'Riots are still going on in Montreal (Canada) because police murdered a boy on Saturday.',b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Torture',b' Russia has just beaten the United States over the head with Peak Oil',b'Perhaps *the* question about the Georgia - Russia conflict ',b'Russia is so much better at war',"b""So this is what it's come to: trading sex for food."""
2,2008-08-12,0,"b'Remember that adorable 9-year-old who sang at the opening ceremonies? That was fake, too.'","b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would have no children...""'","b""Al-Qa'eda is losing support in Iraq because of a brutal crackdown on activities it regards as un-Islamic - including women buying cucumbers""",b'Ceasefire in Georgia: Putin Outmaneuvers the West',b'Why Microsoft and Intel tried to kill the XO $100 laptop',b'Stratfor: The Russo-Georgian War and the Balance of Power ',"b""I'm Trying to Get a Sense of This Whole Georgia-Russia War: Vote Up If You Think Georgia Started It, Or Down If you Think Russia Did""",...,b'U.S. troops still in Georgia (did you know they were in Georgia in the first place?)',b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. 

Import the combined dataset, and place it in a dataframe. This dataset has labels with values 0 - indicating the DJIA went down, and 1 - indicating the DJIA went up or stayed the same. The next 27 columns are the top 27 headlines for a given date in ascending order.

## Clean data
There are a few things we need to do in order to make this data easier to work with. Thankfully, there are no missing values. However, we will want to do some basic text processing in order to get rid of things like stop words, syntax, and more easily vectorize the headlines.

In [109]:
columns = df.iloc[:, 2:]
regx = "[/\-_\+=\*,\.\"\'()!\?@#$%\^;:\[\]]|^b[\'\"]*"
df_no_syntax = df_regx_replace(df_original, columns, regx)
df_no_syntax

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,georgia downs two russian warplanes as countries move to brink of war,breaking musharraf to be impeached,russia today columns of troops roll into south ossetia footage from fighting youtube,russian tanks are moving towards the capital of south ossetia which has reportedly been completely destroyed by georgian artillery fire,afghan children raped with impunity un official says this is sick a three year old was raped and they do nothing,150 russian tanks have entered south ossetia whilst georgia shoots down two russian jets,breaking georgia invades south ossetia russia warned it would intervene on sos side,the enemy combatent trials are nothing but a sham salim haman has been sentenced to 5 12 years but will be kept longer anyway just because they feel like it,...,georgia invades south ossetia if russia gets involved will nato absorb georgia and unleash a full scale war,alqaeda faces islamist backlash,condoleezza rice the us would not act to prevent an israeli strike on iran israeli defense minister ehud barak israel is prepared for uncompromising victory in the case of military hostilities,this is a busy day the european union has approved new sanctions against iran in protest at its nuclear programme,georgia will withdraw 1000 soldiers from iraq to help fight off russian forces in georgias breakaway region of south ossetia,why the pentagon thinks attacking iran is a bad idea us news &amp world report,caucasus in crisis georgia invades south ossetia,indian shoe manufactory and again in a series of you do not like your work,visitors suffering from mental illnesses banned from olympics,no help for mexicos kidnapping surge
1,2008-08-11,1,why wont america and nato help us if they wont help us now why did we help them in iraq,bush puts foot down on georgian conflict,jewish georgian minister thanks to israeli training were fending off russia,georgian army flees in disarray as russians advance gori abandoned to russia without a shot fired,olympic opening ceremony fireworks faked,what were the mossad with fraudulent new zealand passports doing in iraq,russia angered by israeli military sale to georgia,an american citizen living in sossetia blames us and georgian leaders for the genocide of innocent people,...,israel and the us behind the georgian aggression,do not believe tv neither russian nor georgian there are much more victims,riots are still going on in montreal canada because police murdered a boy on saturday,china to overtake us as largest manufacturer,war in south ossetia pics,israeli physicians group condemns state torture,russia has just beaten the united states over the head with peak oil,perhaps the question about the georgia russia conflict,russia is so much better at war,so this is what its come to trading sex for food
2,2008-08-12,0,remember that adorable 9yearold who sang at the opening ceremonies that was fake too,russia ends georgia operation,if we had no sexual harassment we would have no children,alqaeda is losing support in iraq because of a brutal crackdown on activities it regards as unislamic including women buying cucumbers,ceasefire in georgia putin outmaneuvers the west,why microsoft and intel tried to kill the xo 100 laptop,stratfor the russogeorgian war and the balance of power,im trying to get a sense of this whole georgiarussia war vote up if you think georgia started it or down if you think russia did,...,us troops still in georgia did you know they were in georgia in the first place,why russias response to georgia was right,gorbachev accuses us of making a serious blunder in pursuing its interest in the caucasus region,russia georgia and nato cold war two,remember that adorable 62yearold who led your country into war based on evidence that was fake too,war in georgia the israeli connection,all signs point to the us encouraging georgia to invade south ossetia goddamnit bush,c

Removed all puncation/syntax, and lowercased each headline for easier parasing.

In [110]:
df_no_nan = df_text_remove_nan(df_no_syntax)
df_no_nan

removed row: 277
removed row: 348
removed row: 681


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,georgia downs two russian warplanes as countries move to brink of war,breaking musharraf to be impeached,russia today columns of troops roll into south ossetia footage from fighting youtube,russian tanks are moving towards the capital of south ossetia which has reportedly been completely destroyed by georgian artillery fire,afghan children raped with impunity un official says this is sick a three year old was raped and they do nothing,150 russian tanks have entered south ossetia whilst georgia shoots down two russian jets,breaking georgia invades south ossetia russia warned it would intervene on sos side,the enemy combatent trials are nothing but a sham salim haman has been sentenced to 5 12 years but will be kept longer anyway just because they feel like it,...,georgia invades south ossetia if russia gets involved will nato absorb georgia and unleash a full scale war,alqaeda faces islamist backlash,condoleezza rice the us would not act to prevent an israeli strike on iran israeli defense minister ehud barak israel is prepared for uncompromising victory in the case of military hostilities,this is a busy day the european union has approved new sanctions against iran in protest at its nuclear programme,georgia will withdraw 1000 soldiers from iraq to help fight off russian forces in georgias breakaway region of south ossetia,why the pentagon thinks attacking iran is a bad idea us news &amp world report,caucasus in crisis georgia invades south ossetia,indian shoe manufactory and again in a series of you do not like your work,visitors suffering from mental illnesses banned from olympics,no help for mexicos kidnapping surge
1,2008-08-11,1,why wont america and nato help us if they wont help us now why did we help them in iraq,bush puts foot down on georgian conflict,jewish georgian minister thanks to israeli training were fending off russia,georgian army flees in disarray as russians advance gori abandoned to russia without a shot fired,olympic opening ceremony fireworks faked,what were the mossad with fraudulent new zealand passports doing in iraq,russia angered by israeli military sale to georgia,an american citizen living in sossetia blames us and georgian leaders for the genocide of innocent people,...,israel and the us behind the georgian aggression,do not believe tv neither russian nor georgian there are much more victims,riots are still going on in montreal canada because police murdered a boy on saturday,china to overtake us as largest manufacturer,war in south ossetia pics,israeli physicians group condemns state torture,russia has just beaten the united states over the head with peak oil,perhaps the question about the georgia russia conflict,russia is so much better at war,so this is what its come to trading sex for food
2,2008-08-12,0,remember that adorable 9yearold who sang at the opening ceremonies that was fake too,russia ends georgia operation,if we had no sexual harassment we would have no children,alqaeda is losing support in iraq because of a brutal crackdown on activities it regards as unislamic including women buying cucumbers,ceasefire in georgia putin outmaneuvers the west,why microsoft and intel tried to kill the xo 100 laptop,stratfor the russogeorgian war and the balance of power,im trying to get a sense of this whole georgiarussia war vote up if you think georgia started it or down if you think russia did,...,us troops still in georgia did you know they were in georgia in the first place,why russias response to georgia was right,gorbachev accuses us of making a serious blunder in pursuing its interest in the caucasus region,russia georgia and nato cold war two,remember that adorable 62yearold who led your country into war based on evidence that was fake too,war in georgia the israeli connection,all signs point to the us encouraging georgia to invade south ossetia goddamnit bush,c

Remove any rows that contain nan values in the columns.

# Baseline accuracy 
To quickly get a baseline accuracy, combining the columns seems to be the simplest way to get a vectorized data set for classification input. We will do more nuanced vectorization and classification after getting a baseline for this dataset.

## Combine columns

In [131]:
df = df_no_nan
columns = df.iloc[:, 2:]
concat_column = df_concat_str_columns(df, columns)
concat_column = pd.DataFrame(concat_column)
concat_column

,Top1
0,georgia downs two russian warplanes as countries move to brink of war breaking musharraf to be impeached russia today columns of troops roll into south ossetia footage from fighting youtube russian tanks are moving towards the capital of south ossetia which has reportedly been completely destroyed by georgian artillery fire afghan children raped with impunity un official says this is sick a three year old was raped and they do nothing 150 russian tanks have entered south ossetia whilst georgia shoots down two russian jets breaking georgia invades south ossetia russia warned it would intervene on sos side the enemy combatent trials are nothing but a sham salim haman has been sentenced to 5 12 years but will be kept longer anyway just because they feel like it georgian troops retreat from s osettain capital presumably leaving several hundred people killed video did the us prep georgia for war with russia rice gives green light for israel to attack iran says us has no veto over israeli military ops announcingclass action lawsuit on behalf of american public against the fbi sorussia and georgia are at war and the nyts top story is opening ceremonies of the olympics what a fucking disgrace and yet further proof of the decline of journalism china tells bush to stay out of other countries affairs did world war iii start today georgia invades south ossetia if russia gets involved will nato absorb georgia and unleash a full scale war alqaeda faces islamist backlash condoleezza rice the us would not act to prevent an israeli strike on iran israeli defense minister ehud barak israel is prepared for uncompromising victory in the case of military hostilities this is a busy day the european union has approved new sanctions against iran in protest at its nuclear programme georgia will withdraw 1000 soldiers from iraq to help fight off russian forces in georgias breakaway region of south ossetia why the pentagon thinks attacking iran is a bad idea us news &amp world report caucasus in crisis georgia invades south ossetia indian shoe manufactory and again in a series of you do not like your work visitors suffering from mental illnesses banned from olympics no help for mexicos kidnapping surge
1,why wont america and nato help us if they wont help us now why did we help them in iraq bush puts foot down on georgian conflict jewish georgian minister thanks to israeli training were fending off russia georgian army flees in disarray as russians advance gori abandoned to russia without a shot fired olympic opening ceremony fireworks faked what were the mossad with fraudulent new zealand passports doing in iraq russia angered by israeli military sale to georgia an american citizen living in sossetia blames us and georgian leaders for the genocide of innocent people welcome to world war iv now in high definition georgias move a mistake of monumental proportions russia presses deeper into georgia us says regime change is goal abhinav bindra wins first ever individual olympic gold medal for india us ship heads for arctic to define territory drivers in a jerusalem taxi station threaten to quit rather than work for their new boss an arab the french team is stunned by phelps and the 4x100m relay team israel and the us behind the georgian aggression do not believe tv neither russian nor georgian there are much more victims riots are still going on in montreal canada because police murdered a boy on saturday china to overtake us as largest manufacturer war in south ossetia pics israeli physicians group condemns state torture russia has just beaten the united states over the head with peak oil perhaps the question about the georgia russia conflict russia is so much better at war so this is what its come to trading sex for food
2,remember that adorable 9yearold who sang at the opening ceremonies that was fake too russia ends georgia operation if we had no sexual harassment we would have no children alqaeda is losing support in iraq because of a brutal c

Concatenate all the headlines into one corpus to be vectorized in a tdidf/word count vectorizer

In [132]:
df_no_col = df.drop(labels = columns.columns, axis = 1)
df_concat = pd.concat([df_no_col, concat_column], axis = 1)
df_concat

,Date,Label,Top1
0,2008-08-08,0,georgia downs two russian warplanes as countries move to brink of war breaking musharraf to be impeached russia today columns of troops roll into south ossetia footage from fighting youtube russian tanks are moving towards the capital of south ossetia which has reportedly been completely destroyed by georgian artillery fire afghan children raped with impunity un official says this is sick a three year old was raped and they do nothing 150 russian tanks have entered south ossetia whilst georgia shoots down two russian jets breaking georgia invades south ossetia russia warned it would intervene on sos side the enemy combatent trials are nothing but a sham salim haman has been sentenced to 5 12 years but will be kept longer anyway just because they feel like it georgian troops retreat from s osettain capital presumably leaving several hundred people killed video did the us prep georgia for war with russia rice gives green light for israel to attack iran says us has no veto over israeli military ops announcingclass action lawsuit on behalf of american public against the fbi sorussia and georgia are at war and the nyts top story is opening ceremonies of the olympics what a fucking disgrace and yet further proof of the decline of journalism china tells bush to stay out of other countries affairs did world war iii start today georgia invades south ossetia if russia gets involved will nato absorb georgia and unleash a full scale war alqaeda faces islamist backlash condoleezza rice the us would not act to prevent an israeli strike on iran israeli defense minister ehud barak israel is prepared for uncompromising victory in the case of military hostilities this is a busy day the european union has approved new sanctions against iran in protest at its nuclear programme georgia will withdraw 1000 soldiers from iraq to help fight off russian forces in georgias breakaway region of south ossetia why the pentagon thinks attacking iran is a bad idea us news &amp world report caucasus in crisis georgia invades south ossetia indian shoe manufactory and again in a series of you do not like your work visitors suffering from mental illnesses banned from olympics no help for mexicos kidnapping surge
1,2008-08-11,1,why wont america and nato help us if they wont help us now why did we help them in iraq bush puts foot down on georgian conflict jewish georgian minister thanks to israeli training were fending off russia georgian army flees in disarray as russians advance gori abandoned to russia without a shot fired olympic opening ceremony fireworks faked what were the mossad with fraudulent new zealand passports doing in iraq russia angered by israeli military sale to georgia an american citizen living in sossetia blames us and georgian leaders for the genocide of innocent people welcome to world war iv now in high definition georgias move a mistake of monumental proportions russia presses deeper into georgia us says regime change is goal abhinav bindra wins first ever individual olympic gold medal for india us ship heads for arctic to define territory drivers in a jerusalem taxi station threaten to quit rather than work for their new boss an arab the french team is stunned by phelps and the 4x100m relay team israel and the us behind the georgian aggression do not believe tv neither russian nor georgian there are much more victims riots are still going on in montreal canada because police murdered a boy on saturday china to overtake us as largest manufacturer war in south ossetia pics israeli physicians group condemns state torture russia has just beaten the united states over the head with peak oil perhaps the question about the georgia russia conflict russia is so much better at war so this is what its come to trading sex for food
2,2008-08-12,0,remember that adorable 9yearold who sang at the opening ceremonies that was fake too russia ends georgia operation if we had no sexual harassment we would have no children alqae

Remove redundant, non-combined columns and append the new concatenated column onto the existing dataframe with labels.

## Vectorize data 

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
sw = stop.ENGLISH_STOP_WORDS
tfidf = TfidfVectorizer(stop_words = sw)
tfidf.fit(df_concat['Top1'])
print(tfidf.get_feature_names()[0:1000])

['000', '0001', '001', '003', '004', '005', '006', '007', '0077', '00s', '01', '011', '014', '014ckwh', '017', '02', '0200', '021hour', '022', '0220', '0221', '025', '03', '030', '035017', '04', '05', '050', '05eurgb', '06', '060', '062', '068as', '07', '07232014', '075', '07baku1268', '07pc', '08', '089m', '08baku671', '09', '0900', '0930', '0935', '10', '100', '1000', '10000', '100000', '1000000', '1000000000', '100000man', '100000person', '100000th', '10000s', '10000strong', '10000yearold', '10001500', '1000km', '1000megawatt', '1000per', '1000s', '1000strong', '1000x', '1000year', '1000yearold', '100200', '1004', '1004am', '1006', '100abarrel', '100apack', '100billion', '100bn', '100day', '100ds', '100fold', '100foot', '100ft', '100k', '100km', '100m', '100mil', '100million', '100mstretch', '100mw', '100page', '100s', '100th', '100x', '100year', '100yearold', '101', '1011', '1012', '1017', '101st', '102', '1020', '102000', '1021', '10262010', '102day', '102yearold', '103', '1030', 

Create a dictionary in a tfidf vectorizer given our concatenated dataset.

In [134]:
features_con = tfidf.transform(df_concat['Top1'])
data_labels = df_concat['Label']
display(features)
print(features)

<1986x39900 sparse matrix of type '<class 'numpy.float64'>'
	with 489780 stored elements in Compressed Sparse Row format>

  (0, 39697)	0.04792113397841143
  (0, 39586)	0.018635470668102998
  (0, 39576)	0.02309480578615759
  (0, 39317)	0.03510663510162514
  (0, 39295)	0.03222955083293025
  (0, 39198)	0.052627215370409826
  (0, 38962)	0.07472527781990451
  (0, 38644)	0.05458685467845757
  (0, 38632)	0.03848278340226787
  (0, 38591)	0.09580465919607202
  (0, 38346)	0.05874965900421731
  (0, 38220)	0.028760672420553638
  (0, 38214)	0.04589661170480536
  (0, 38159)	0.0641100318305066
  (0, 37476)	0.07190063735605241
  (0, 37430)	0.03528136571540746
  (0, 37203)	0.08424845497195356
  (0, 36718)	0.057575186629517626
  (0, 36629)	0.055135230165267744
  (0, 36160)	0.0639523086984654
  (0, 35872)	0.05955281974015128
  (0, 35600)	0.035923795876247486
  (0, 35374)	0.09332647229079422
  (0, 34955)	0.05458685467845757
  (0, 34709)	0.04748600345927687
  :	:
  (1985, 3536)	0.09229817581704898
  (1985, 3108)	0.0404703132373173
  (1985, 3009)	0.03580320373466395
  (1985, 2796)	0.06189488586264328
  (1985, 2761)	0.035896283

We have converted the *whole* column of concatenated headlines into a single sparse matrix using the tfidf transformer built on the dictionary created earlier. At this point, I'm not sure if I should have created a sparse matrix for each row, or use the above method instead.

## Training/test sets

In [135]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    features_con,
    data_labels,
    train_size = 0.80,
    random_state = 1234
)
display(x_train)
display(x_test)

c:\python\installation\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


<1588x39900 sparse matrix of type '<class 'numpy.float64'>'
	with 392171 stored elements in Compressed Sparse Row format>

<398x39900 sparse matrix of type '<class 'numpy.float64'>'
	with 97609 stored elements in Compressed Sparse Row format>

Partition the whole concatenated dataset into seperate train/test sets.

## Begin training

### Naive Bayes

In [136]:
from sklearn.naive_bayes import GaussianNB
gnbc = GaussianNB()
gnbc.fit(x_train.toarray(), y_train)
print(make_percent(gnbc.score(x_test.toarray(), y_test)))

46.98%


### Decision Tree

In [138]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train.toarray(), y_train)
print(make_percent(dtc.score(x_test.toarray(), y_test)))

49.25%


### Logistic Regression

In [139]:
from sklearn.linear_model import LogisticRegression
lgc = LogisticRegression()
lgc.fit(x_train.toarray(), y_train)
print(make_percent(lgc.score(x_test.toarray(), y_test)))

51.01%
